<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/project/model1_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
@File name: model1_cross_validation.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import math

import numpy as np

from keras import optimizers
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization, Activation
from keras.callbacks import EarlyStopping

from sklearn.utils import compute_class_weight
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

import sys
sys.path.append('local_modules')

import postprocessing_sw

In [0]:
#fix random seed for reproducibility
seed = 5
np.random.seed(seed)

In [0]:
'''
@Description: generate a multilayer perceptron with LeakyRelu as activation
function.
@param: 
  - shape : int, the shape of the input
  - n_features: int, the number of features given
'''

def generate_model_leaky(shape, n_features):
  
  units_1 = int(n_features/2)
  units_2 = n_features
  units_3 = n_features*2
  
  model = Sequential()
  model.add(BatchNormalization())
  
  model.add(Dense(units_1, input_dim=shape, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(units_2, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(units_3, kernel_initializer='random_uniform',  use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model

In [0]:
#FUNZIONE AGGIORNATA
def get_precision(p, r):
  index_pred = []
  index_real = []

  prediction_meals = []
  real_meals = []

  #p = flat_pred
  #r = test_label

  #Costruzione delle liste di indici: una di pasti predetti e una di pasti reali

  for i in range(0, len(p)):
    if p[i] == 1:
      index_pred.append(i)
      if ((i+1) in range(0, len(p)) and p[i+1] == 0) or (i+1) not in range(0, len(p)): 
        if(len(index_pred)>5):        
          prediction_meals.append(index_pred)
        index_pred = []
    if r[i] == 1:
      index_real.append(i)
      if ((i+1) in range(0, len(p)) and r[i+1] == 0) or (i+1) not in range(0, len(p)): 
        real_meals.append(index_real)
        index_real = []

        
  #Ricerca delle intersezioni

  intersection = [] #contiene 1 se per un pasto reale j è stata già segnata un'intersezione 
  tp = 0
  fp = 0

  for i in range(0, len(real_meals)): #inizializzazione a 0 per l'array di intersezioni
    intersection.append(0)

  for i in range(0, len(prediction_meals)): #per ogni pasto predetto
    flag_found = 0 #per tenere conto se l'intersezione è già stata trovata all'interno dei pasti reali
    count_int = 0 #tiene conto del numero di intersezioni che un pasto predetto ha con i pasti reali (per il caso in cui un pasto pred intersechi due pasti reali)

    for j in range(0, len(real_meals)):# per ogni pasto reale    
      flag_visited = 0 #tiene conto se un pasto di prediction meals è già stato analizzato (per il caso in cui due pasti pred intersechino un unico pasto reale)

      for x in prediction_meals[i]: #per ogni minuto di pasto controllo se esiste un'intersezione dentro real meals

        if x in real_meals[j]: #se c'è l'intersezione        

          if intersection[j]==0: #ciò significa che non è mai stata trovata un intersezione per il pasto reale j-esimo

            if count_int == 0: #controlla che non siano ancora state trovate intersezioni  
              count_int = count_int + 1
              intersection[j] = 1
              flag_visited = 1 #imposta il flag a uno per indicare che il pasto reale è stato già analizzato e per non contare più di una volta il tp trovato
              flag_found = 1 #imposta il flag a uno per indicare che il pasto predetto è stato trovato nei pasti reali
              tp=tp+1

            elif count_int > 0: #caso in cui un pasto pred interseca due pasti reali e quindi un tp è già stato trovato in precedenza (produce un falso positivo e un vero positivo)           
              intersection[j] = 1
              flag_visited = 1 #imposta il flag a uno per indicare che il pasto reale è stato già analizzato e per non contare più di una volta il tp e il fp trovati
              flag_found = 1            
              fp=fp+1
              tp=tp+1

          elif flag_visited == 0: #caso in cui l'intersezione per il pasto reale j-esimo è già stata trovata ma tale pasto non è ancora stato analizzato con il pasto predetto i-esimo
                                  #risolve il caso in cui più pasti pred intersecano un unico pasto reale
            flag_found = 1
            flag_visited = 1
    #end for j

    if flag_found == 0: #se l'intersezione nn è stata trovata nei pasti reali il pasto predetto è falso positivo
      fp=fp+1
    
    
  #Ricerca falsi negativi

  fn = 0

  for i in range(0, len(real_meals)): #per ogni pasto reale
    j=0
    flag_found = 0
    while j in range(0, len(prediction_meals)) and flag_found==0: # per ogni pasto predetto finchè non viene trovato tra questi l'intersezione con il pasto i-esimo

      for x in real_meals[i]: #per ogni minuto di pasto controllo se esiste un'intersezione dentro prediction_meals

        if x in prediction_meals[j]: #se c'è l'intersezione        
          flag_found = 1 #imposta il flag a uno per indicare che il pasto reale è stato trovato nei pasti predetti

      j = j + 1  
    #end while j

    if flag_found == 0: #se l'intersezione del pasto reale non è stata trovata nei pasti predetti siamo in corrispondenza di un falso negativo
      fn=fn+1

  #Calcolo accuratezza
  print("N° pasti reali:", len(real_meals))
  print("N° pasti predetti:", len(prediction_meals))

  print("TP:", tp)
  print("FP:", fp)
  print("FN:", fn)

  somma = tp+fp
  precision = (tp/(tp+fp))*100
  TPR = (tp/(tp+fn))*100
  F1 = 2*((precision*TPR)/(precision+TPR))
  
  print("Recall: %.2f %%" % TPR)
  print("Precisione: %.2f %%" % precision)
  print("F1 score: %.2f %%" % F1)

In [0]:
#@title CHOOSE

'''
@Description: MAIN
'''

#LOAD DATA
print("Loading data...")

dataset = '/root/data/uHooComplete_featureDataset_Past_Only.arff' #@param {type:"string"}

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("\nConverting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

n_features = s-1

#OPTIMIZERS
adm = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#LOSS
loss = 'binary_crossentropy'

#DEFINE K-FOLD CROSS-VALIDATION
fold = 10 #@param {type:"integer"}

kfold = KFold(n_splits=fold, shuffle=False, random_state=None)

cvscores = []
predictions = []

true_p =[]
true_n = []
precision = []
fscore= []

test_time =[]
real = []

dimSplit = math.floor(len(dataset[:,0])/10)
startIndex = 0
finishIndex = dimSplit-1

i = 1

for train, test in kfold.split(X, Y):
  
  print("\nFOLD: %d" %i)
  
  #COMPUTE CLASS WEIGHT
  labels = np.unique(Y[train])
  classWeight = compute_class_weight('balanced', labels, Y[train])
  classWeight = dict(zip(labels,classWeight))

  #GENERATE MODEL
  print("\nGenerate model...")

  model = generate_model_leaky(X[train].shape[-1], n_features)

  #COMPILE MODEL
  print("\nCompile model...")
  model.compile(loss = loss, optimizer = adm , metrics=['accuracy'])

  #FIT MODEL
  print("\nFit model...")
  epochs = 8 #@param{type:"integer"}
  history = model.fit(X[train], Y[train], epochs=epochs, batch_size = 128, shuffle = True, verbose=1, class_weight = classWeight)

  #EVALUATE MODEL
  print("\nEvaluate model...")
  scores_test = model.evaluate(X[test], Y[test], batch_size= 128, verbose = 0)
  print("Test loss: %.2f%%" % (scores_test[0] * 100))
  print("Test accuracy: %.2f%%" % (scores_test[1] * 100))
  
  cvscores.append(scores_test[1] * 100)
  
  #CALCULATE PREDICTIONS
  print("\nCalculate predictions...")
  pred = model.predict_classes(X[test], batch_size=128, verbose=0)
  flat_pred = [item for sublist in pred for item in sublist]
  predictions.append(flat_pred)
  real.append(Y[test])
  
  #STORE DATETIME
  time = []  
  for j in X[test]:
    time.append(int(j[-5]))  
  test_time.append(time)
  
  #CONFUSION MATRIX
  print("\nCompute confusion matrix...")
  y_true = Y[test]
  tn, fp, fn, tp = confusion_matrix(y_true, pred).ravel()
  tnr = 100*tn/(tn+fp)
  tpr = 100*tp/(fn+tp)
  prec = 100*(tp/(tp+fp))
  F1 = 2*((prec*tpr)/(prec+tpr))
  print("TRUE NEGATIVE RATE (SPECIFICITY): %.2f %%" % tnr)
  print("TRUE POSITIVE RATE (RECALL): %.2f %%" % tpr)
  print("PRECISION: %.2f %%" % prec)
  print("F1 SCORE: %.2f %%" % F1)
  
  true_p.append(tpr)
  true_n.append(tnr)
  precision.append(prec)
  fscore.append(F1)
  
  i+=1

print("MEAN ACCURACY: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("MEAN TRUE POSITIVE RATE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(true_p), np.std(true_p)))
print("MEAN TRUE NEGATIVE RATE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(true_n), np.std(true_n)))
print("MEAN PRECISION: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(precision), np.std(precision)))
print("MEAN F1 SCORE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(fscore), np.std(fscore)))

#FLATTENING LISTS
flat_predictions = [item for sublist in predictions for item in sublist]
true_y = [arr.tolist() for arr in real]
flat_true_y = [item for sublist in true_y for item in sublist]
flat_time = [item for sublist in test_time for item in sublist]


#CONFUSION MATRIX BEFORE POST PROCESSING
print("\nCompute confusion matrix BEFORE POST PROCESSING...")
tn, fp, fn, tp = confusion_matrix(flat_true_y, flat_predictions).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
tnr = 100*tn/(tn+fp)
tpr = 100*tp/(fn+tp)
precision = 100*(tp/(tp+fp))
F1 = 2*((precision*tpr)/(precision+tpr))
print("TRUE NEGATIVE RATE (SPECIFICITY): %.2f %%" % tnr)
print("TRUE POSITIVE RATE (RECALL): %.2f %%" % tpr)
print("PRECISION: %.2f %%" % precision)
print("F1 SCORE: %.2f %%" % F1)


In [0]:
#POST PROCESSING
new_pred =  postprocessing_sw.sliding_windows(flat_predictions,33)

#CONFUSION MATRIX AFTER POST PROCESSING
print("\nCompute confusion matrix AFTER POST PROCESSING...")
tn, fp, fn, tp = confusion_matrix(flat_true_y, new_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
tnr = 100*tn/(tn+fp)
tpr = 100*tp/(fn+tp)
precision = 100*(tp/(tp+fp))
F1 = 2*((precision*tpr)/(precision+tpr))
print("TRUE NEGATIVE RATE (SPECIFICITY): %.2f %%" % tnr)
print("TRUE POSITIVE RATE (RECALL): %.2f %%" % tpr)
print("PRECISION: %.2f %%" % precision)
print("F1 SCORE: %.2f %%" % F1)

In [0]:
get_precision(flat_predictions,flat_true_y)


In [0]:
get_precision(new_pred, flat_true_y)